In [1]:
import pandas as pd
import json
import os
import numpy as np
import requests
import sqlalchemy as sql
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from pathlib import Path
import datetime
%matplotlib inline

In [2]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set response URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"
ltc_url = "https://api.alternative.me/v2/ticker/Litecoin/?convert=USD"

In [4]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response,indent=4,sort_keys=True))

{
    "data": {
        "1": {
            "circulating_supply": 18635218,
            "id": 1,
            "last_updated": 1613960969,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 1058889486262,
                    "percent_change_1h": 0.345224346693775,
                    "percent_change_24h": 0.549658322203009,
                    "percent_change_7d": 17.6724206477506,
                    "percentage_change_1h": 0.345224346693775,
                    "percentage_change_24h": 0.549658322203009,
                    "percentage_change_7d": 17.6724206477506,
                    "price": 57198.0,
                    "volume_24h": 61696223124
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 18635218,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrenci

In [5]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response,indent=4,sort_keys=True))

{
    "data": {
        "1027": {
            "circulating_supply": 114769779,
            "id": 1027,
            "last_updated": 1613960870,
            "max_supply": 0,
            "name": "Ethereum",
            "quotes": {
                "USD": {
                    "market_cap": 219117775760,
                    "percent_change_1h": -0.356256815075714,
                    "percent_change_24h": -2.62148050263032,
                    "percent_change_7d": 5.84654238236493,
                    "percentage_change_1h": -0.356256815075714,
                    "percentage_change_24h": -2.62148050263032,
                    "percentage_change_7d": 5.84654238236493,
                    "price": 1910.51,
                    "volume_24h": 32634391620
                }
            },
            "rank": 2,
            "symbol": "ETH",
            "total_supply": 114769779,
            "website_slug": "ethereum"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurr

In [6]:
# Using the Python requests library, make an API call to access the current price LTC
ltc_response = requests.get(ltc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(ltc_response,indent=4,sort_keys=True))

{
    "data": {
        "2": {
            "circulating_supply": 66528753,
            "id": 2,
            "last_updated": 1613960928,
            "max_supply": 84000000,
            "name": "Litecoin",
            "quotes": {
                "USD": {
                    "market_cap": 14948477440,
                    "percent_change_1h": 0.275809314917534,
                    "percent_change_24h": -3.10997821171731,
                    "percent_change_7d": 4.94541468451555,
                    "percentage_change_1h": 0.275809314917534,
                    "percentage_change_24h": -3.10997821171731,
                    "percentage_change_7d": 4.94541468451555,
                    "price": 225.65,
                    "volume_24h": 8440114373
                }
            },
            "rank": 8,
            "symbol": "LTC",
            "total_supply": 66528753,
            "website_slug": "litecoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrencies

In [7]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response['data']['1']['quotes']['USD']['price']

In [8]:
# Navigate the ETH response object to access the current price of ETH
eth_price = eth_response['data']['1027']['quotes']['USD']['price']

In [9]:
# Navigate the LTC response object to access the current price of LTC
ltc_price = ltc_response['data']['2']['quotes']['USD']['price']

In [10]:
# remember to update date
today_date = str(datetime.datetime.now()).split()[0]
one_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=1*365)).split()[0]
three_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=3*365)).split()[0]
str(datetime.datetime.now()).split(" ")[0]
# Format current date as ISO format
today = pd.Timestamp(today_date, tz="America/New_York").isoformat()
today
# Format start and end dates as ISO format for one year period
start = pd.Timestamp(one_year_ago, tz="America/New_York").isoformat()
end = pd.Timestamp(today_date, tz="America/New_York").isoformat()
# Included for example
# # Get closing prices for FB and TWTR from the last year
# df_portfolio_year = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = start,
#     end = end
# ).df

In [11]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='v2')
display(type(alpaca_api_key)) # test to make sure api key imported successfully, str indicates import successful
display(type(alpaca_secret_key)) # test to make sure api key imported successfully, str indicates import successful

str

str

In [12]:
# # Set the tickers to be used
# large_company_stocks=['QQQ']

# medium_company_stocks=['ARKK']

# small_company_stocks=[
# 'XSTLA',
# 'CLF',
# 'MSTR',
# 'NEO',
# 'DDD',
# 'OMCL']

# bonds = ['AGG']

In [13]:
tickers = ['AGG','QQQ']

# Set timeframe to 1D 
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
start_date = pd.Timestamp('2016-02-19',tz='America/New_York').isoformat() # remember to update date
end_date = pd.Timestamp('2021-02-19',tz='America/New_York').isoformat() # remember to update date

In [14]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_barset(tickers,timeframe,start=start_date,end=end_date).df

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

AGG                                         QQQ  \
                             open    high       low   close   volume     open   
2020-09-28 00:00:00-04:00  118.18  118.22  118.1100  118.20  5318716  276.615   
2020-09-29 00:00:00-04:00  118.28  118.33  118.2100  118.23  3974292  276.720   
2020-09-30 00:00:00-04:00  118.26  118.26  117.9516  118.07  5314447  276.090   
2020-10-01 00:00:00-04:00  117.81  118.04  117.7500  117.99  9535086  281.800   
2020-10-02 00:00:00-04:00  118.03  118.04  117.8100  117.92  5356203  276.010   

                                                                 
                               high       low   close    volume  
2020-09-28 00:00:00-04:00  277.0400  271.6740  277.03  51184391  
2020-09-29 00:00:00-04:00  277.7100  275.3719  275.86  23663696  
2020-09-30 00:00:00-04:00  281.0900  275.8717  278.10  44849000  
2020-10-01 00:00:00-04:00  282.8794  279.8400  282.26  44554843  
2020-10-02 00:00:00-04:00  282.2412  273.4400  274.33  66448911

In [15]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
# agg_close_price = prices_df['AGG']['close'].item()

In [16]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
# spy_close_price = prices_df['SPY']['close'].item()

In [17]:
# Access the closing price for QQQ from the Alpaca DataFrame
# Converting the value to a floating point number
# qqq_close_price = prices_df['QQQ']['close'].item()

In [18]:
# Access the closing price for ARKK from the Alpaca DataFrame
# Converting the value to a floating point number
# arkk_close_price = prices_df['ARKK']['close'].item()

In [19]:
# Access the closing price for O from the Alpaca DataFrame
# Converting the value to a floating point number
# o_close_price = prices_df['O']['close'].item()

In [20]:
# Access the closing price for VNQ from the Alpaca DataFrame
# Converting the value to a floating point number
# vnq_close_price = prices_df['VNQ']['close'].item()
# print(vnq_close_price)

In [21]:
# Get user info
user_info = pd.read_csv(Path('data/user_info.csv'))

In [22]:
# Start to build portfolio options
user_info

,current_age,yearly_retirement_income,retirement_age,current_savings,retirement_goal,portfolio_allocation
0,23,50000,65,500000,1250000,"[0.0, 0.7, 0.3]"


In [23]:
age = user_info['current_age']

In [24]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user doesn't want crypto
user_info.loc[:,'portfolio_allocation'] = user_info.loc[:,'portfolio_allocation'].str.replace("'","")
portfolio_allocation = str(user_info['portfolio_allocation'])
print(portfolio_allocation)

0    [0.0, 0.7, 0.3]
Name: portfolio_allocation, dtype: object


In [25]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user does want crypto
if portfolio_allocation == '0    [0.0, 0.7, 0.3]' or '0    [0.2,0.6,0.2]' or '0    [0.4,0.4,0.2]' or '0    [0.5,0.4,0.1]' or '0    [0.7,0.2,0.1]':
    percent_bonds = portfolio_allocation[7:9]
else:
    percent_bonds = portfolio_allocation[6:8]
print(percent_bonds)

.0


In [26]:
if portfolio_allocation == '0    [0.0, 0.7, 0.3]' or '0    [0.2,0.6,0.2]' or '0    [0.4,0.4,0.2]' or '0    [0.5,0.4,0.1]' or '0    [0.7,0.2,0.1]':
    percent_stocks = portfolio_allocation[12:14]
else:
    percent_stocks = portfolio_allocation[10:12]
print(percent_stocks)

.7


In [27]:
if portfolio_allocation == '0    [0.0, 0.7, 0.3]' or '0    [0.2,0.6,0.2]' or '0    [0.4,0.4,0.2]' or '0    [0.5,0.4,0.1]' or '0    [0.7,0.2,0.1]':
    percent_crypto = portfolio_allocation[17:19]
else:
    percent_crypto = portfolio_allocation[14:16]
print(percent_crypto)

.3


In [28]:
stock_amt = float(percent_stocks)*user_info['current_savings']
stock_amt

0    350000.0
Name: current_savings, dtype: float64

In [29]:
btc_amt = float(percent_crypto)*(user_info['current_savings'])
print(btc_amt)

0    150000.0
Name: current_savings, dtype: float64


In [30]:
bond_amt = float(percent_bonds)*(user_info['current_savings'])
print(bond_amt)

0    0.0
Name: current_savings, dtype: float64


In [31]:
if float(stock_amt) > 0:
    stock_weight = (float(stock_amt)+float(bond_amt))/float(stock_amt)
else:
    stock_weight = 0
print(stock_weight)

1.0


In [32]:
if float(bond_amt) > 0:
    bond_weight = (float(bond_amt)+float(stock_amt))/float(bond_amt)
else:
    bond_weight = 0
print(bond_weight)

0


In [33]:
portfolio_weights = [bond_weight,stock_weight]
print(portfolio_weights)

[0, 1.0]


In [35]:
MC_sim = (MCSimulation(portfolio_data=prices_df,weights=portfolio_weights,num_simulation=500,num_trading_days=252*30))
MC_sim.portfolio_data.head()

AGG                                     \
                             open    high       low   close   volume   
2020-09-28 00:00:00-04:00  118.18  118.22  118.1100  118.20  5318716   
2020-09-29 00:00:00-04:00  118.28  118.33  118.2100  118.23  3974292   
2020-09-30 00:00:00-04:00  118.26  118.26  117.9516  118.07  5314447   
2020-10-01 00:00:00-04:00  117.81  118.04  117.7500  117.99  9535086   
2020-10-02 00:00:00-04:00  118.03  118.04  117.8100  117.92  5356203   

                                            QQQ                              \
                          daily_return     open      high       low   close   
2020-09-28 00:00:00-04:00          NaN  276.615  277.0400  271.6740  277.03   
2020-09-29 00:00:00-04:00     0.000254  276.720  277.7100  275.3719  275.86   
2020-09-30 00:00:00-04:00    -0.001353  276.090  281.0900  275.8717  278.10   
2020-10-01 00:00:00-04:00    -0.000678  281.800  282.8794  279.8400  282.26   
2020-10-02 00:00:00-04:00    -0.000593  276.010  282.2412  273.4400  274.33   

                                                  
                             volume daily_return  
2020-09-28 00:00:00-04:00  51184391          NaN  
2020-09-29 00:00:00-04:00  23663696    -0.004223  
2020-09-30 00:00:00-04:00  44849000     0.008120  
2020-10-01 00:00:00-04:00  44554843     0.014959  
2020-10-02 00:00:00-04:00  66448911    -0.028095

In [36]:
MC_sim.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.010755,0.996318,1.023470,1.000561,1.013595,9.866537e-01,0.995569,1.012636,0.995695,1.012943,...,0.983473,1.005885e+00,1.024285e+00,1.005564,0.985417,0.993558,0.997156,0.990099,0.998093,1.021357
2,1.028840,1.016650,1.029944,1.001789,1.027942,1.000540e+00,1.006684,1.004974,0.994247,1.011865,...,1.005808,1.015008e+00,1.023981e+00,0.991809,0.999237,1.015556,1.009480,1.010592,1.010263,1.039411
3,1.026609,1.019807,1.024924,0.980908,1.041819,1.000581e+00,0.983664,1.003976,0.981976,1.002854,...,1.016236,1.049724e+00,1.002980e+00,0.985772,0.997589,1.017033,1.016512,0.987359,1.018367,1.045900
4,1.035009,1.026214,1.012926,0.983388,1.013455,9.983720e-01,0.983314,0.989971,0.987688,0.989037,...,1.035763,1.054417e+00,9.951362e-01,0.987872,1.005334,1.005979,1.000960,0.978473,1.025534,1.056185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,204263.695676,635377.661080,167423.126856,188692.461194,494518.987051,1.073771e+06,288004.737692,445708.096932,760709.954041,198771.751390,...,752744.936257,1.374573e+06,1.769097e+06,320302.808261,143810.586721,146209.112612,298327.417611,417823.944330,300798.273686,195328.291621
7557,200255.196052,642854.471097,166422.611412,191229.780966,499333.107934,1.100509e+06,291884.215338,446178.337182,748969.432084,203117.743721,...,748923.847173,1.367756e+06,1.784810e+06,319210.990938,142899.120750,145883.003771,299883.553770,418729.058559,300952.966030,192419.964971
7558,205324.413902,645650.275990,162462.437417,193948.710011,504682.264913,1.085614e+06,289912.647570,449484.231626,757302.235770,205781.873777,...,765715.560432,1.356238e+06,1.762048e+06,317924.948737,145647.658625,143550.014242,293682.349757,414359.395477,308772.820727,195755.220843
7559,203676.337838,649496.819540,163197.423425,193438.598583,498467.918983,1.062730e+06,294161.321783,448036.886647,751042.467067,202193.052899,...,762239.667317,1.343303e+06,1.775071e+06,315671.324250,147061.332679,144903.907050,292720.760363,420777.932511,301955.839583,196133.671096


In [38]:
cumulative_returns = MC_sim.summarize_cumulative_return()
print(cumulative_returns)

count           5.000000e+02
mean            1.407475e+06
std             2.001898e+06
min             2.617743e+04
25%             3.834546e+05
50%             7.879145e+05
75%             1.620729e+06
max             2.370469e+07
95% CI Lower    8.142562e+04
95% CI Upper    6.499388e+06
Name: 7560, dtype: float64


In [ ]:
# GOAL: finish MVP by Tuesday, full by Thursday
# Build portfolios with each allocation
# Run monte carlos and graphs
# Finish CLI
# Test for bugs
# 